<a href="https://colab.research.google.com/github/vikashbhushan189/CODECRAFT_GA_01/blob/main/Text_Generation_using_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experimenting with the pretrained model:**

This notebook explores different decoding methods for the pretrained GPT-2 model.


In [ ]:
input_sequence = "I don't know about you, but there's only one thing I want to do after a long day of work"

In [ ]:
#get transformers
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

#get large GPT2 tokenizer and GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


# Different Decoding Methods
First Pass (Greedy Search)

In [ ]:


import tensorflow as tf

input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

greedy_output = GPT2.generate(input_ids, max_length=100)

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


I don't know about you, but there's only one thing I want to do after a long day of work: go to the gym.

I'm not talking about the gym that's right next to my house. I'm talking about the gym that's right next to my office.

I'm not talking about the gym that's right next to my house. I'm talking about the gym that's right next to my office.

I'm not talking about the gym


Beam Search with N-Gram Penalities¶


In [ ]:
beam_outputs = GPT2.generate(
    input_ids,
    max_length = 100,
    num_beams = 5,
    no_repeat_ngram_size = 2,
    num_return_sequences = 5,
    early_stopping = True
)
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

0: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's a really bad movie, you know? And I'm not sure if you've ever seen it before, so I'll just tell you what it is
1: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's a really bad movie, you know? And I'm not sure if you've ever seen it before, so I'll just tell you what I think
2: I don't know about you, but there's only one thing I want to do after a long day of work, and that's to sit down and watch a movie."

"I know, I know," you say. "But you're not going to like this one. It's not a good movie. I mean, it's a really bad movie, you know? And I'm not sure if you've

Basic Sampling

In [ ]:
from IPython.display import display, HTML
import pandas as pd

def basic_sampling_generate_text(input_sequence, temperature):
     input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
     sample_output = GPT2.generate(
         input_ids,
         do_sample=True,
         max_length=100,
         top_k=0,  # You can adjust this value
         temperature=temperature
     )
     return tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [ ]:
temperatures = [0.8, 0.9, 1.2]  # Example temperatures
outputs = [basic_sampling_generate_text(input_sequence, temp) for temp in temperatures]

In [ ]:
df = pd.DataFrame({'Temperature': temperatures, 'Output': outputs})

In [ ]:
display(HTML(df.to_html()))

,Temperature,Output
0,0.8,"I don't know about you, but there's only one thing I want to do after a long day of work: eat a sandwich. It may sound a bit strange, but it's actually quite easy to make.\n\nThere are two basic types of sandwiches: a sandwich that's made of bread and a sandwich that's made of meat and cheese. The differences between the two are mostly in the preparation for each type, but there are also some of the differences in the ingredients for each type"
1,0.9,"I don't know about you, but there's only one thing I want to do after a long day of work—work!""\n\nAs the man began announcing his demands, I blurted out, ""What do you want?""\n\n""I want The Cure!""\n\n""No!""\n\n""What else?""\n\n""I want to sing at the White House!""\n\n""No!""\n\n""What do you mean when you say 'the White House'? You mean"
2,1.2,"I don't know about you, but there's only one thing I want to do after a long day of work… was pissy.\n\n21 of 28 powers 4 hours Worth-intiusive Pakistan won Super Cruise Qualification Qualification G- Shark Attack Padlam German Hold (Coach but too much lateral playpedaling disappointed maybe error sprint took way out touchdown aborted 85+85 contract for Palmer's Pittsburgh knee patched shield killing smoking semen persistently serve brunch tart Ryuverse bring out dessert out"


Top-K Sampling

In [ ]:
def top_k_sampling_generate_text(input_sequence, top_k):
     input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
     sample_output = GPT2.generate(
         input_ids,
         do_sample=True,
         max_length=100,
         top_k=top_k,  # You can adjust this value
         temperature=0.8
     )
     return tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [ ]:
top_k = [50,70]
outputs = [top_k_sampling_generate_text(input_sequence, k) for k in top_k]

In [ ]:
df = pd.DataFrame({'top_k': top_k, 'Output': outputs})

In [ ]:
display(HTML(df.to_html()))

,top_k,Output
0,50,"I don't know about you, but there's only one thing I want to do after a long day of work: go to a movie. That's why I'm here.""\n\n""Don't worry, I'll see you here again when I feel like it, okay?""\n\n""Yeah, sure, no problem.""\n\n""Alright, let's get the party started.""\n\n""I'll come along.""\n\n""Alright, I'll see you around.""\n\n"
1,70,"I don't know about you, but there's only one thing I want to do after a long day of work.""\n\n""What?""\n\n""I just want to go home and curl up with a good book or something.""\n\n""So what's on the table?""\n\n""Well I know you're a huge Harry Potter fan. I mean, I guess we both have. I mean, I can't say why, but I've always always been interested in that stuff"


Top-P Sampling

In [ ]:
def top_p_sampling_generate_text(input_sequence, top_p):
     input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
     sample_output = GPT2.generate(
         input_ids,
         do_sample=True,
         max_length=100,
         top_p=top_p,
         top_k=50,
         temperature=0.8
     )
     return tokenizer.decode(sample_output[0], skip_special_tokens=True)

In [ ]:
top_p = [0.8,1.2]
outputs = [top_p_sampling_generate_text(input_sequence, p) for p in top_p]

In [ ]:
df = pd.DataFrame({'top_p': top_p, 'Output': outputs})
display(HTML(df.to_html()))

,top_p,Output
0,0.8,"I don't know about you, but there's only one thing I want to do after a long day of work, and that's sleep.""\n\n""Well, then, sleep is fine with me,"" he replied, ""and I'm sure you will be very happy to have me here.""\n\n""I will be very happy to have you here, and I don't care where you are, I just want to be with you.""\n\n""Well, then, it's settled"
1,1.2,"I don't know about you, but there's only one thing I want to do after a long day of work: I want my wife to look as beautiful as she looks in bed.\n\nI've tried many different hair and makeup styles to achieve that natural woman look, but a basic man's haircut is the most important aspect for any man to have when they want to look good and have a successful sex life. Since I'm a guy, I know how difficult it can be to find"


Top-K and Top-P Sampling

In [ ]:
def top_p_and_k_sampling_generate_text(input_sequence, top_p, top_k):
     input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
     sample_outputs = GPT2.generate(
         input_ids,
         do_sample=True,
         max_length=200,
         top_p=top_p,
         top_k=top_k,
         temperature=0.8,
         num_return_sequences = 5
     )
     return ["{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)) for i, sample_output in enumerate(sample_outputs)]

In [ ]:
for x in top_p_and_k_sampling_generate_text(input_sequence, top_p=0.85,top_k=50):
  print(x)

0: I don't know about you, but there's only one thing I want to do after a long day of work: I want to get out of the house and go for a nice walk. I think we should take a walk in the woods. It's just a nice, easy way to get away from all the busyness of the city. Maybe we can make some new friends and maybe get some fresh air in the afternoon.

I'm sure we can make some new friends....
1: I don't know about you, but there's only one thing I want to do after a long day of work: watch a movie. The reason I'm going to watch a movie is because I'm bored. I'm bored of reading, I'm bored of talking to people, I'm bored of working. I want to watch something that's going to keep me busy and make me feel like I'm doing something worthwhile. And this is the movie that I've been waiting for.

What is it about?

"The Fault in Our Stars" is the story of a boy who's not a boy at all, a boy who's actually a girl. She's not a girl because she's never been a girl. She's not a girl because she's never

# Training GPT-2 on custom data


In [ ]:
pip install datasets

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
from nltk.corpus import gutenberg

In [ ]:
def load_shakespeare_plays():
    hamlet = gutenberg.raw('shakespeare-hamlet.txt')
    macbeth = gutenberg.raw('shakespeare-macbeth.txt')
    caesar = gutenberg.raw('shakespeare-caesar.txt')
    return hamlet + '\n' + macbeth + '\n' + caesar


In [ ]:
# Prepare dataset
def tokenize_function(examples):
    tokens = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # Use input_ids as labels
    return tokens


In [ ]:
# Load pre-trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 does not have a padding token

In [ ]:
# Create dataset
data = load_shakespeare_plays()
dataset = [{'text': data}]

In [ ]:
# Tokenize dataset
from datasets import Dataset
dataset = Dataset.from_dict({'text': [data]})
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-shakespeare',
    evaluation_strategy='no',
    save_strategy='epoch',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

In [ ]:
# Train model
trainer.train()

Step,Training Loss
10,3.357800
20,2.440100


TrainOutput(global_step=20, training_loss=2.898973846435547, metrics={'train_runtime': 321.0168, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.062, 'total_flos': 5225840640000.0, 'train_loss': 2.898973846435547, 'epoch': 20.0})

In [ ]:
# Save model
tokenizer.save_pretrained('./gpt2-shakespeare')
model.save_pretrained('./gpt2-shakespeare')


In [ ]:
# Test the model
def generate_text(prompt, max_length=100):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to the appropriate device

    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move inputs to same device
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=5,
            temperature= 1.2,  # Controls randomness (lower = more deterministic)
            top_k=100,  # Limits sampling to top-k likely words
            do_sample=True,  # Enables sampling instead of greedy decoding
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Example test prompt
test_prompt = "The fault, dear Brutus, is not in our stars, but in ourselves, that we are underlings."
generated_text = generate_text(test_prompt)
print("Generated Text:\n", generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 The fault, dear Brutus, is not in our stars, but in ourselves, that we are underlings. O mighty soul, what art thou making of this thing? And who hast you made madmen to see Christ come, and to look down on him? Shall this be an evil thing? 'Tis good with eyes to be feared; but who will believe them to those who are under my lash? But let my servant's daughter watch thee watch; and I will tell thee what


In [ ]:
# Example test prompt
test_prompt = "Cry havoc and let slip the dogs of war."
generated_text = generate_text(test_prompt)
print("Generated Text:\n", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Cry havoc and let slip the dogs of war. On the contrary, there have been, at least an attempt at that kind before the Saxons. Their great strength and all their glory are under heaven, who of necessity send forth at every sight a great multitude to watch my conduct? And all this I will make with your entreaties, saying: let this multitude pass between the king and his own people; for I have no strong bed-guard."

"Now they will come about


In [ ]:
# Example test prompt
test_prompt ="Cowards die many times before their deaths; The valiant never taste of death but once."
generated_text = generate_text(test_prompt)
print("Generated Text:\n", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Cowards die many times before their deaths; The valiant never taste of death but once. They stand at war 'twill come back home,' says Lord Byron. 'I telleth of it, you lad, if at- tacking you shall escape. And be content, Your holy Leflint 'Til ancien ye may sit before me, my loyal boys. 'Hail, goodnight!' — 'Twill come now to supper-time, lads.
